# Accounts Information Provider 
In this toy example, I will create a brochure that takes information about bank products but focuses only on account-based products and creates a one-paragraph summary of each account type. The workflow is as follows: the LLM consumes PDF files with product descriptions, works only on products related to accounts, summarizes each into a single paragraph, and then compiles them into a brochure.

## Import Required Python Packages

In [66]:
import os
from pathlib import Path
from openai import OpenAI
from pathlib import Path
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from diskcache import Cache
from IPython.display import Markdown, display, update_display


## Initialize DiskCache
This avoids calling LLMs on same inputs 

In [ ]:

cache = Cache("./cache/llm_engineering_week_1_accounts_info_provider")

## Setup Workspace and Other Constants

In [54]:
# Setup Input folders 
DIR_WORKSPACE = Path.cwd().parents[0]
DIR_DATA = DIR_WORKSPACE / "data"
DIR_BANK_PRODUCT_PROFILES   = DIR_DATA / "docs" / "national-bank-products"

# Test Documents Paths
FILE_TEST_PDF_ACC_PRODUCT = DIR_BANK_PRODUCT_PROFILES / "Amayi Angathe Business Savings Account.pdf"
FILE_TEST_PDF_NON_ACC_PRODUCT = DIR_BANK_PRODUCT_PROFILES / "224-electronic-funds-transfer-fact-sheet.pdf"

# OpenAI API Key
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Ensure API key is available
if OPENAI_API_KEY is None:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

# Initialise Disk Cache
CACHE = Cache("./cache/llm_engineering_week_1_accounts_info_provider")

# Default OpenAI Model
DEFAULT_OPENAI_MODEL = "gpt-4.1-mini"

## Function to Load PDFs
This section defines a function to load and read PDF files from the specified directory containing bank product profiles.

In [4]:
def load_pdf(path):
    
    reader = PdfReader(path)
    text = ""
    
    for page in reader.pages:
        text += page.extract_text()
    
    return text

In [11]:
# Lets load a test PDF document
pdf_text = load_pdf(FILE_TEST_PDF_ACC_PRODUCT)
print(pdf_text[:500])  # Print the first 500 characters of the PDF text

Amayi Angathe Business Savings AccountAmayi Angathe Business Savings AccountAmayi Angathe Business Savings AccountAmayi Angathe Business Savings AccountThe Amayi Angathe Product and Service Offering has three main categories. These are; a Savings Account, Credit Facility andan Information Web Portal with dedicated Account Relationship Managers. The attributes of this product and serviceoffering are summarised below:AMAYI ANGATHE BUSINESS SAVINGS ACCOUNTAMAYI ANGATHE BUSINESS SAVINGS ACCOUNTAMAYI


## Select Account Related Products
First, we need to identify and filter documents that describe account products. This step uses the LLM to classify whether a product document is account-related or not, ensuring we only process relevant documents for the brochure.

### Define a System Prompt

In [20]:
account_classifier_system_prompt = """
You are an assistant that analyzes PDF documents containing bank product descriptions.
Your task is to determine whether the document describes an account product (savings account, 
checking account, current account, business account, etc.) or a non-account product 
(loans, credit cards, insurance, payment services, etc.).

You should respond in JSON format with a boolean field indicating if it's an account product.

Example:
Input: "Premium Business Current Account - Designed for growing businesses, this account offers 
unlimited transactions, dedicated relationship manager, and preferential interest rates on balances 
above MK 100,000. Minimum opening balance of MK 50,000."

Output:
{
    "is_account_product": true,
    "product_name": "Premium Business Current Account",

Now analyze the provided document and respond in the same JSON format.
"""

In [16]:
def messages_product_classifier(pdf_text):
    return [
        {"role": "system", "content": account_classifier_system_prompt},
        {"role": "user", "content": f"Here is the document to analyze:\n\n{pdf_text}"}
    ]

### Test Product Classification

In [55]:
# Test PDF document classification-Account Product
pdf_text_account = load_pdf(FILE_TEST_PDF_ACC_PRODUCT)
messages = messages_product_classifier(pdf_text_account)

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model=DEFAULT_OPENAI_MODEL,
    messages=messages,
    temperature=0.2,
    max_tokens=500
)

print("Response for Account Product Document:")
print(response.choices[0].message.content)

Response for Account Product Document:
{
  "is_account_product": true,
  "product_name": "Amayi Angathe Business Savings Account"
}


In [56]:
# Test PDF document classification-Non-Account Product
pdf_text_non_account = load_pdf(FILE_TEST_PDF_NON_ACC_PRODUCT)
messages = messages_product_classifier(pdf_text_non_account)

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model=DEFAULT_OPENAI_MODEL,
    messages=messages,
    temperature=0.2,
    max_tokens=500
)

print("Response for Account Product Document:")
print(response.choices[0].message.content)

Response for Account Product Document:
{
    "is_account_product": false,
    "product_name": "Electronic Funds Transfer"
}


## Classify All Accounts 

In [57]:
classification_result = []

for pdf_file in DIR_BANK_PRODUCT_PROFILES.glob("*.pdf"):
    cache_key = f"{DEFAULT_OPENAI_MODEL}_{pdf_file.resolve()}"  # Include model in cache key

    # ✅ Check cache first
    if cache_key in CACHE:
        result = CACHE[cache_key]
        print(f"[CACHE HIT] {pdf_file.name}")
    else:
        print(f"[LLM CALL] {pdf_file.name}")

        pdf_text = load_pdf(pdf_file)
        messages = messages_product_classifier(pdf_text)

        response = client.chat.completions.create(
            model=DEFAULT_OPENAI_MODEL,
            messages=messages,
            temperature=0.2,
            max_tokens=80  # classification → keep this small
        )

        result = response.choices[0].message.content

        # ✅ Save to cache
        CACHE[cache_key] = result

    classification_result.append((str(pdf_file.resolve()), result))


[LLM CALL] 221-student-serve-account.pdf
[LLM CALL] 230-documentary-letter-of-credit-factsheet.pdf
[LLM CALL] Banknet 360 Fact Sheet.pdf
[LLM CALL] 304-diaspora-account.pdf
[LLM CALL] 286-step-up-fact-sheet.pdf
[LLM CALL] 219-savings-account.pdf
[LLM CALL] 235-consumer-loan-factsheet.pdf
[LLM CALL] 239-vehicle-loans-factsheet.pdf
[LLM CALL] 233-amayi-angathe-factsheet.pdf
[LLM CALL] 237-mortgage-home-improvement-factsheet.pdf
[LLM CALL] 222-banknet-360-fact-sheet.pdf
[LLM CALL] 231-open-account-factsheet.pdf
[LLM CALL] 218-mlimi-account.pdf
[LLM CALL] 220-special-savers-account.pdf
[LLM CALL] 212-call-accounts.pdf
[LLM CALL] 213-current-account.pdf
[LLM CALL] 240-correspondent-banks-factsheet.pdf
[LLM CALL] 292-special-package-for-accountants.pdf
[LLM CALL] 215-fixed-term-deposit.pdf
[LLM CALL] Amayi Angathe Business Savings Account.pdf
[LLM CALL] 227-visa-debit-card-fact-sheet.pdf
[LLM CALL] 236-employer-guaranteed-factsheet.pdf
[LLM CALL] 225-mo626-digital-fact-sheet.pdf
[LLM CALL] 2

## Product Profile Summarization

### Product Summarization System Prompt

In [58]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are an assistant that analyzes the contents of a PDF document,
and provides a short, summary, ignoring text that might be navigation related. Provide a summary equivalent 
to 1 paragraph focusing on key details of the product described in the document.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

### Product Summarization User Prompt

In [59]:
user_prompt_prefix = """
Here are the contents of a PDF describing a bank product:
Provide a short summary of this product in markdown format.
{pdf_text}
"""

In [34]:
def messages_summary(pdf_text):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix.format(pdf_text=pdf_text)}
    ]

### Test Product Summarization

In [60]:
for pdf_file, classification in classification_result:
    # We only want to summarize account products
    if '"is_account_product": true' in classification:
        print(f"Summarizing: {pdf_file}")

        cache_key = f"{DEFAULT_OPENAI_MODEL}_summary_{pdf_file}"
        
        # Check cache first
        if cache_key in CACHE:
            summary = CACHE[cache_key]
            print(f"[CACHE HIT] {Path(pdf_file).name}")
        else:
            print(f"[LLM CALL] {Path(pdf_file).name}")
            
            pdf_text = load_pdf(pdf_file)
            messages = messages_summary(pdf_text)

            response = client.chat.completions.create(
                model=DEFAULT_OPENAI_MODEL,
                messages=messages,
                temperature=0.2,
                max_tokens=500
            )

            summary = response.choices[0].message.content
            
            # Save to cache
            CACHE[cache_key] = summary

        print(f"Summary for {Path(pdf_file).name}:\n{summary}\n")

Summarizing: /Users/dmatekenya/GIT-REPOS/llm_engineering/data/docs/national-bank-products/221-student-serve-account.pdf
[LLM CALL] 221-student-serve-account.pdf
Summary for 221-student-serve-account.pdf:
The Student Serve Account is designed for students aged 18 and above, requiring a minimum balance of MK500 and offering an interest rate of 2.5% per annum, subject to review. Key benefits include a free Visa Debit Card and no service fees on personal accounts. There are no costs to access the product. To open the account, students must provide account opening forms, a valid student ID from a recognized higher education institution, a signature specimen or thumbprint, utility bills, and a reference letter from their school. The account must not be overdrawn or fall below the minimum balance of MK500.

Summarizing: /Users/dmatekenya/GIT-REPOS/llm_engineering/data/docs/national-bank-products/304-diaspora-account.pdf
[LLM CALL] 304-diaspora-account.pdf
Summary for 304-diaspora-account.pdf:

In [62]:
def get_account_product_summaries():
    """
    Generate summaries for all account products and return them as a concatenated string.
    
    Returns:
        str: A concatenated string of all account product summaries
    """
    summaries = []
    
    for pdf_file, classification in classification_result:
        # We only want to summarize account products
        if '"is_account_product": true' in classification:
            cache_key = f"{DEFAULT_OPENAI_MODEL}_summary_{pdf_file}"
            
            # Check cache first
            if cache_key in CACHE:
                summary = CACHE[cache_key]
                print(f"[CACHE HIT] {Path(pdf_file).name}")
            else:
                print(f"[LLM CALL] {Path(pdf_file).name}")
                
                pdf_text = load_pdf(pdf_file)
                messages = messages_summary(pdf_text)

                response = client.chat.completions.create(
                    model=DEFAULT_OPENAI_MODEL,
                    messages=messages,
                    temperature=0.2,
                    max_tokens=500
                )

                summary = response.choices[0].message.content
                
                # Save to cache
                CACHE[cache_key] = summary
            
            summaries.append(summary)
    
    # Concatenate all summaries with double newlines for separation
    return "\n\n".join(summaries)

In [ ]:
# Get all account summaries as a single string
all_summaries = get_account_product_summaries()
print(f"\nTotal summaries: {len(all_summaries.split(chr(10)+chr(10)))}")
print("\nCombined summaries ready for brochure generation:")
print(all_summaries)

## Generate Brochure

### Brochure System Prompt 

In [47]:
brochure_system_prompt = """
You are an assistant that creates marketing brochures for banking products.
Your task is to take summaries of multiple account products and create a professional, 
engaging brochure that highlights the bank's account offerings for prospective customers.

The brochure should:
- Have a compelling introduction about the bank's account portfolio
- Present each account product in a clear, organized manner
- Highlight key benefits and features of each account
- Use professional yet accessible language
- Be formatted in markdown without code blocks

Respond only with the markdown content of the brochure.
"""

In [49]:
def get_brochure_user_prompt(bank_name, account_summaries):
    user_prompt = f"""
Create a professional marketing brochure for {bank_name}'s account products.

Below are summaries of all our account offerings. Use this information to create an engaging, 
well-organized brochure that highlights the key features and benefits of each account type.

Account Product Summaries:

{account_summaries}

Please create a comprehensive brochure that presents these accounts in an appealing way for prospective customers.
"""
    return user_prompt

In [64]:
def create_brochure(bank_name, summaries):
    response = client.chat.completions.create(
        model=DEFAULT_OPENAI_MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(bank_name, summaries)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [67]:
create_brochure("National Bank", all_summaries)

# National Bank Account Portfolio  
**Empowering Your Financial Journey at Every Stage**

At National Bank, we understand that your financial needs are as unique as your ambitions. Whether you are a student embarking on your academic journey, a young professional climbing the career ladder, a thriving entrepreneur, or a seasoned business owner, our diverse account offerings are designed to support your goals with tailored benefits, innovative services, and trusted banking solutions.

Explore our comprehensive range of accounts, thoughtfully crafted to help you save, invest, transact, and grow confidently.

---

## Personal Accounts

### Student Serve Account  
**Banking Tailored for Students Aged 18+**  
- **Minimum Balance:** MK500  
- **Interest Rate:** 2.5% p.a. (subject to review)  
- **Key Benefits:**  
  - Free Visa Debit Card  
  - No service fees on personal accounts  
  - Zero access costs  
- **Ideal For:** Students enrolled in recognized higher education institutions  
- **Features:** Requires student ID and school reference for account opening; overdraft not permitted; maintain minimum balance to stay in good standing.

---

### ITF Account  
**Secure Savings for Children and Minors**  
- **Minimum Balance:** MK5,000  
- **Interest Rate:** 3% p.a. (subject to review)  
- **Key Benefits:**  
  - Guardian-managed account with fund protection  
  - No access fees  
- **Ideal For:** Parents and guardians saving for children’s future needs  
- **Features:** Funds transferred to child at adulthood; non-inclusion in guardian’s estate if deceased before child turns 18.

---

### Special Savers Account  
**Affordable Savings for Employees and Business Owners**  
- **Minimum Balance:** MK1,000  
- **Interest Rate:** 1% p.a. (subject to review)  
- **Key Benefits:**  
  - Low minimum balance requirement  
  - Affordable monthly charges  
- **Note:** E-service and ATM card fees apply; account must not be overdrawn.

---

### Savings Account  
**Reliable Savings for Individuals and Businesses**  
- **Minimum Balance:** MK5,000  
- **Interest Rate:** 3% p.a. (subject to review)  
- **Key Benefits:**  
  - Competitive interest rates paid annually  
  - No service fees on personal accounts   
- **Features:** Accessible via ATM, POS, withdrawal slips; e-service and ATM card fees apply.

---

### FIESTASAVE Account  
**Save for Life’s Special Moments and Wealth Accumulation**  
- **Minimum Opening Balance:** MK20,000  
- **Monthly Minimum Deposit:** MK10,000  
- **Interest Rate:** 6% higher than ordinary savings accounts  
- **Key Benefits:**  
  - No service fees  
  - Use your savings as collateral  
  - Quarterly withdrawal options (over the counter)  
- **Ideal For:** Personal and SME customers planning weddings, vacations, or major investments.

---

### Call Account  
**Flexible Short-Term Investment with Competitive Returns**  
- **Minimum Balance:** MK50,000  
- **Interest Rates:**  
  - 7-day call account: 2.5% p.a.  
  - 30-day call account: 2.25% p.a.  
- **Key Benefits:**  
  - No bank charges  
  - Can be used as security for borrowing  
  - Flexible access through sweep account attachment  
- **Note:** Interest forfeited on early withdrawal before maturity.

---

### Current Account  
**Convenient Daily Transactions for All Your Payment Needs**  
- **Minimum Balance:** None  
- **Key Benefits:**  
  - Cheque book facility for secure payments  
  - Overdraft access for qualifying clients  
  - No service fees on personal accounts  
- **Ideal For:** Personal, SME, and corporate clients  
- **Note:** Applicable charges include e-service fees, cheque book fees, and ATM card fees.

---

### Foreign Currency Denominated Account (FCDA)  
**Manage Your Foreign Currency with Confidence**  
- **Minimum Balance:** None  
- **Key Benefits:**  
  - Negotiable interest rates  
  - Ability to hedge against currency fluctuations  
  - Withdraw local currency via Malawi Kwacha linked operating account  
- **Ideal For:** Exporters, Malawians abroad, and those earning in foreign currency  
- **Compliance:** Operates in line with Reserve Bank of Malawi Exchange Control Regulations.

---

## Specialized Accounts for Farmers, Groups & Clubs

### MLIMI Account  
**Simple Savings for Farmers, Clubs, and Estates**  
- **Minimum Balance:** MK200  
- **Key Benefits:**  
  - Zero access costs  
  - Supports financial management for farmers  
- **Ideal For:** Individual farmers, clubs, and estate owners  
- **Features:** Requires proof of farmer status.

---

### Diaspora Account  
**Seamless Banking for Malawians Living Abroad**  
- **Currencies:** Malawi Kwacha & Foreign Currencies  
- **Key Benefits:**  
  - No e-service fees, free internet banking  
  - Visa debit and credit cards  
  - Special access to Diaspora Fixed-Term Deposit Account  
  - Higher credit interest rate (+2%) on MWK savings  
  - Dedicated relationship support  
  - Borrowing options including mortgages and loans  
  - Real estate partnerships & funeral insurance benefits  
- **Ideal For:** Individuals, clubs, and societies abroad  
- **Opening Requirements:** Passport, referee details, embassy/immigration verification.

---

## Professional & Entrepreneur Accounts

### Step-Up Professional Account  
**Empowering Young Professionals (18-35 Years)**  
- **Eligibility:** Minimum Bachelor’s degree; monthly net salary MK300,000 - MK500,000  
- **Minimum Balance:** MK10,000  
- **Interest Rate:** Savings interest 2% above normal rates  
- **Key Benefits:**  
  - Reduced monthly fees  
  - Visa Debit Card with Account Relationship Manager  
  - Upgrade to Premium Gold account at MK500,000 salary  
  - Discounted digital banking (Mo626 Digital+, BankNet360)  
  - Loans for education, professional fees, vehicles, and more (up to MK5 million)  
  - Financial management and wealth creation workshops  
- **Credit Terms:** Interest at Reference Rate +8%

---

### Step Up Entrepreneur Account  
**Comprehensive Solutions for Young Entrepreneurs (21-35 Years)**  
- **Eligibility:** Business account holder; monthly turnover ≤ MK8.4 million  
- **Key Benefits:**  
  - Discounted transactional account fees  
  - Working capital loans up to MK5 million (2-year terms)  
  - Invoice financing up to MK10 million  
  - Unsecured guarantees & bonds up to MK5 million  
  - Credit card limit of MK1 million for travel & e-payments  
  - Access to start-up capital via innovation symposium  
  - Business management workshops and digital banking support  
- **Credit Terms:** Interest at Reference Rate +7.1%; 1% arrangement fee  
- **Collateral:** No collateral required for credit facilities

---

## Business & Corporate Accounts

### Amayi Angathe Business Savings Account  
**Supporting Lady-Owned Businesses to Thrive**  
- **Eligibility:** Annual turnover up to MK50,000,000  
- **Minimum Balance:** MK3,000  
- **Key Benefits:**  
  - No service fees for first 6 months; then concessionary fee MK200  
  - Free ATM card and Mo626 plus ATM usage  
  - Discounted salary payment fees (EFT)  
  - Credit bundle with loans up to MK5 million and credit cards up to MK1 million  
  - Repayment up to 24 months at concessionary interest rates  
  - 1% arrangement fee, no collateral required  
  - Access to exclusive web portal with business tools and peer support  
- **Ideal For:** Women entrepreneurs focused on business growth

---

### Fixed Term Deposit Account  
**Secure Your Funds with Attractive Fixed Returns**  
- **Minimum Balance:** MK25,000  
- **Interest:** High rates paid at end of term; compounded for renewed periods  
- **Key Benefits:**  
  - Very low risk investment  
  - No bank charges  
  - Usable as security for borrowing  
- **Note:** Early withdrawals result in forfeiture of interest.

---

# Why Bank with National Bank?  
- **Customized Solutions:** Accounts designed to suit your lifestyle and financial goals.  
- **Competitive Interest Rates:** Maximize your savings and investments.  
- **Advanced Digital Banking:** Enjoy convenient, secure access via Mo626 Digital+, BankNet360, and internet banking.  
- **Dedicated Support:** Relationship managers and tailored services to guide your journey.  
- **Inclusive Banking:** From students and farmers to entrepreneurs and diaspora clients, we serve your unique needs.

---

**Ready to open your account?**  
Visit any National Bank branch or our website to learn more and get started with the financial solution that fits you best.

**National Bank** — Your Partner in Progress and Prosperity.

In [70]:
def stream_brochure(bank_name, summaries):
    stream = client.chat.completions.create(
        model=DEFAULT_OPENAI_MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(bank_name=bank_name, account_summaries=summaries)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [71]:
stream_brochure("National Bank", all_summaries)

# Welcome to National Bank: Your Partner in Financial Success

At National Bank, we understand that your financial needs are as unique as your goals. Whether you are a student starting out, a young professional building your career, an entrepreneur growing your business, or someone planning for the future, we offer a comprehensive portfolio of banking products designed with you in mind. Our suite of accounts provides flexible solutions, competitive rates, and personalized services to empower you every step of the way.

Explore our diverse range of accounts and discover how National Bank can support your financial journey today.

---

## Personal Accounts

### Student Serve Account
**Designed for**: Students aged 18+  
**Minimum Balance**: MK500  
**Interest Rate**: 2.5% p.a. (subject to review)  

**Key Benefits:**  
- Free Visa Debit Card  
- No service fees on personal accounts  
- No costs to access the account  
- Easy overdraft protection by maintaining minimum balance  
- Tailored for students with flexible account opening requirements including student ID and school references  

Start your financial journey the smart way with an account built to support your studies and lifestyle.

---

### Step-Up Professional Account
**Designed for**: Young salaried professionals aged 18–35 with a minimum bachelor’s degree  
**Minimum Balance**: MK10,000  
**Interest Rate**: Savings 2% above normal rates  

**Key Benefits:**  
- Reduced monthly fees and discounted digital banking services (Mo626 Digital+, BankNet360)  
- Access to Visa Debit Card and dedicated Account Relationship Manager  
- Upgrade to Premium Gold upon reaching MK500,000 salary  
- Loan facilities for education, professional fees, vehicles, and more (up to MK5 million total exposure)  
- Financial management and wealth creation workshops  

Grow your career with banking designed to enhance your financial wellbeing and professional aspirations.

---

### Savings Account
**Designed for**: Individuals and corporate organizations  
**Minimum Balance**: MK5,000  
**Interest Rate**: 3% p.a., paid annually (subject to review)  

**Key Benefits:**  
- Competitive interest rate for steady growth  
- Easy access via ATM, POS, and withdrawal slips  
- No service fees on personal accounts (e-service and ATM fees apply)  
- Suitable for personal and business savings  

Secure your savings with an account that balances accessibility and competitive returns.

---

### Special Savers Account
**Designed for**: Salaried employees, business owners, employed individuals  
**Minimum Balance**: MK1,000  
**Interest Rate**: 1% p.a. (subject to review)  

**Key Benefits:**  
- Low minimum balance to encourage saving habits  
- Affordable monthly charges  
- Accessible savings with required KYC documentation  

Ideal for those starting to build their savings with an account tailored for convenience and modest returns.

---

### Call Account
**Designed for**: Customers seeking short-term investments  
**Minimum Balance**: MK50,000  
**Interest Rate**: 2.5% p.a. (7-day call), 2.25% p.a. (30-day call) (subject to review)  

**Key Benefits:**  
- No bank charges  
- Funds can be used as security for borrowing  
- Attached to sweep accounts for easy access and flexible cash management  
- Interest forfeiture applies on early withdrawal, encouraging disciplined saving  

Optimize your short-term funds with flexibility and attractive returns.

---

### Fixed Term Deposit
**Designed for**: Customers with surplus funds for fixed-term investments  
**Minimum Balance**: MK25,000  
**Interest Rate**: High, paid at maturity and compounded thereafter  

**Key Benefits:**  
- Very low risk, no bank charges  
- Can be used as borrowing security  
- Encourages disciplined, goal-driven savings  
- Early withdrawal results in forfeited interest, reinforcing commitment  

Grow your wealth securely with guaranteed returns over a fixed period.

---

### ITF Account (In Trust For)
**Designed for**: Children and minors under 18  
**Minimum Balance**: MK5,000  
**Interest Rate**: 3% p.a. (subject to review)  

**Key Benefits:**  
- Managed by guardian signatory until the minor reaches adulthood  
- Funds excluded from guardian’s estate in case of their demise  
- Account transfers directly to the minor at 18  
- No access costs, encourages early savings habits  

Plan for your child’s future with a secure and transparent savings account.

---

### FIESTASAVE Account
**Designed for**: Personal and SME customers  
**Minimum Balance**: MK20,000 (opening), monthly deposits ≥ MK10,000  
**Interest Rate**: 6% higher than ordinary savings  

**Key Benefits:**  
- No service fees  
- Tailored for wealth accumulation and special events  
- Account can be used as collateral for loans  
- Withdrawals allowed quarterly over the counter  

Achieve your dreams faster by saving smart for life’s biggest moments.

---

## Business & Specialized Accounts

### Amayi Angathe Business Savings Account  
**Designed for**: Lady-owned businesses with annual turnover ≤ MK50 million  
**Minimum Balance**: MK3,000  
**Monthly Fees**: MK200, waived first 6 months  

**Key Benefits:**  
- Free ATM card and Mo626 Plus ATM usage  
- Discounted electronic funds transfer fees for salary payments  
- Access to a credit bundle (loans up to MK5 million, credit cards up to MK1 million) with concessionary interest and no collateral required  
- Exclusive web portal featuring business tools, resources, and events for lady entrepreneurs  

Empowering women entrepreneurs with finance and community support for sustainable growth.

---

### Step Up Entrepreneur Package  
**Designed for**: Young entrepreneurs aged 21–35 with active business banking relationship  
**Turnover Limit**: Max average monthly turnover of MK8.4 million  

**Key Benefits:**  
- Discounted transactional banking fees  
- Working capital loans up to MK5 million (2-year term), order/invoice financing up to MK10 million  
- Unsecured guarantees and bonds up to MK5 million per transaction  
- Credit card facility up to MK1 million for travel and e-payments  
- Startup support including innovation symposium access  
- Business management workshops and digital banking services  
- Collateral-free credit options  

Fuel your entrepreneurial ambitions with flexible finance and expert guidance at every stage.

---

### Diaspora Account  
**Designed for**: Malawians living or working abroad (individuals, clubs, societies)  

**Key Benefits:**  
- Transactional accounts in Malawi Kwacha and foreign currencies  
- No e-service fees and free internet banking  
- Visa debit and credit cards  
- Access to Diaspora Fixed-Term Deposits and preferential interest rates (2% above normal)  
- Borrowing options including mortgages, home improvements, asset and consumer loans  
- Investment opportunities and funeral insurance partnerships  
- Dedicated relationship support and strict compliance with Reserve Bank of Malawi regulations  

Stay connected and financially empowered, wherever you are in the world.

---

### Foreign Currency Denominated Account (FCDA)  
**Designed for**: Exporters, foreign income earners, Malawians abroad  

**Key Benefits:**  
- No minimum balance requirement  
- Withdrawals via linked Malawi Kwacha operating account  
- Negotiable interest rates on foreign currency holdings  
- Hedge against currency fluctuations  
- Compliant with RBM Exchange Control Regulations  

Manage your foreign currency with confidence and convenience.

---

### Current Account  
**Designed for**: Personal, SME, and corporate clients  

**Key Benefits:**  
- No minimum balance requirement  
- Access to cheque book facilities and overdraft options  
- No service fees on personal accounts  
- Secure cheque payments compliant with banking regulations  
- Suitable for day-to-day transactional needs  

Experience flexible and convenient banking solutions for your everyday business and personal transactions.

---

### MLIMI Account  
**Designed for**: Clubs, farmers, estate owners  

**Minimum Balance**: MK200  

**Key Benefits:**  
- Simple, no-cost savings solution for agricultural communities  
- Supports effective management of farming proceeds  
- Easy account opening with proof of farming status  

Supporting Malawi’s backbone—empowering farmers to secure their financial future.

---

# Choose National Bank Today

At National Bank, we are committed to delivering banking solutions that match your lifestyle and ambitions. Our wide array of accounts offers everything from starter packages for students and young professionals to specialized products for entrepreneurs, farmers, and diaspora customers. With competitive rates, convenient digital services, personalized support, and a firm focus on customer protection, National Bank stands ready to help you reach your financial goals.

**Visit your nearest branch or our website to open your ideal account today. National Bank — Banking Made for You.**